<a href="https://colab.research.google.com/github/shadiakiki1986/ml-competitions/blob/master/other/201902-gym-wtp/WtpComboEnv_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WTP combo designer + operator

This jupyter notebook demonstrates an openai/gym environment simulating the combined task of design and operation of a water treatment plant. It also trains a feed-forward neural network to achive the goal of design of an optimal system to treat a randomly sampled set of water parameters as well as to control the process.

The design elements are the same as in the `WtpDesigner_v0` notebook: turbidity, hardness, UV

The operation elements are the same as in the `WtpOperator_v0` notebook: pump, bypasses

The goal is to transfer water from a source tank to a product tank and treating it along the way.

The simulation combines design and operation by performing a 1st stage of design, followed by a 2nd stage of operation.

Note that as of 2019-02-16, this is still WIP.

# install pre-reqs

In [1]:
# install openai gym
!pip install gym | tail
!pip show gym

Name: gym
Version: 0.10.11
Summary: The OpenAI Gym: A toolkit for developing and comparing your reinforcement learning agents.
Home-page: https://github.com/openai/gym
Author: OpenAI
Author-email: gym@openai.com
License: UNKNOWN
Location: /usr/local/lib/python3.6/dist-packages
Requires: scipy, numpy, requests, six, pyglet
Required-by: tensor2tensor, stable-baselines, dopamine-rl


In [2]:
# install dependencies of rlworkgroup/garage
# Copied from colab/2019-01-21/t3.ipynb
#------------------------------------

# Install dependencies (copied from garage/environment.yml)
!apt-get install libglfw3 libglfw3-dev | tail

# >>>>>>>>   requires restart of runtime in colab.research.google.com due to joblib and rsa <<<<<<
!pip install awscli  boto3  cached_property  cloudpickle  cma==1.1.06 flask  gym  "box2d-py>=2.3.4"  hyperopt  ipdb  ipywidgets  jsonmerge  "joblib<0.13,>=0.12"  jupyter  mako  matplotlib  memory_profiler  pandas  path.py    polling  pre_commit  protobuf  psutil  pygame  pyglet  PyOpenGL  pyprind  python-dateutil  pyzmq  scikit-image  scipy  tensorboard  | tail
#"tensorflow<1.10,>=1.9.0"  Theano==1.0.2    "mujoco-py<1.50.2,>=1.50.1" gym[all]==0.10.8
#!pip install jsonmerge glfw mako pygame
!pip install pyprind cma glfw | tail

# Install garage (continued in next cell)
!git clone https://github.com/rlworkgroup/garage
# !cd garage && pip install -e . # >>>>>>>>   requires restart of runtime in colab.research.google.com due to joblib and rsa <<<<<<
#!pip show rlgarage garage

Reading package lists...
Building dependency tree...
Reading state information...
libglfw3 is already the newest version (3.2.1-1).
libglfw3-dev is already the newest version (3.2.1-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
fatal: destination path 'garage' already exists and is not an empty directory.


In [3]:
# Install garage, commit e7324a68dedd94b4ea15a9c761bab2af032e2480 before the upcoming commits related to bumping gym/dm-control/mujoco versions
!cd garage && git checkout e7324a68dedd94b4ea15a9c761bab2af032e2480
!cd garage && pip install -e . # >>>>>>>>   requires restart of runtime in colab.research.google.com due to joblib and rsa <<<<<<

HEAD is now at e7324a6 Move nb_utils.py to garage.experiment
Obtaining file:///content/garage
  Found existing installation: rlgarage 0.1.0
    Can't uninstall 'rlgarage'. No files were found to uninstall.
  Running setup.py develop for rlgarage




---



---



---



# Utility functions

In [0]:
import pandas as pd
import numpy as np

In [0]:
# mappings between integer indices
#-----------------------------

# number of elements in a single WTP
n_elements = 5 # 1 # 5 # 10 causes too big of an action space (check unflatten action class)

#-----------------------------

# state
state_keys = [
    # wl: water level in raw water tank
    "wl_in",
]

# append water parameters for each element
for i in range(n_elements):
  sk_ = [
    # water parameters at input of element i
    "e%i_turbidity"%i, # float 0,100
    "e%i_hardness"%i, # float 0,100
    "e%i_bacteria"%i, # float 0,100
    # pd: pressure difference
    "e%i_pd"%i, # float 0,100
    # type of element (from element_types array below)
    # moved here from action_keys
    "e%i_type"%i, # integer 0,1,2,3
  ]
  state_keys += sk_
  
# append more
state_keys += [
    # water level in product tank
    "wl_out",
    
    # water parameters of product tank
    "out_turbidity",
    "out_hardness",
    "out_bacteria",
    
    # 1 for design phase, 0 for operation phase .. supposed to help the agent identify a change of "era"
    "design_mode",
]

#-----------------------------
# keys to action tuple
action_keys = [
    "pump_status", # manually putting a pump at the start of this sequence
    "ei_type", # since action bit to sequentially yield a sequence of WTP elements
]
for i in range(n_elements):
  action_keys += [
    # type of element (from element_types array below)
    #"e%i_type"%i, # integer 0,1,2,3

    # For a pump, status=off <=> pump is off
    # For a sand filter, being off = bypass is open
    "e%i_status"%i, # boolean 0,1
  ]

#-----------------------------
# mapping of element int to string
element_types = [
    "pipe",
    "sand filter",
    "softener",
    "UV",
    # no need for bypass ATM since "sand filter off" = "sand filter bypass open"
    # "bypass"
]
#-----------------------------

# elements uses
element_uses = {
    "sand filter": "turbidity",
    "softener": "hardness",
    "UV": "bacteria",
}


#state_keys, action_keys

In [0]:
def convert_lists_ObsActRew_df(solution):
    """
    Utility function for display of results
    """
    # blend into a single dataframe
    solution['act'] = pd.DataFrame(solution['act'])
    solution['obs'] = pd.DataFrame(solution['obs'])
    solution = pd.concat([solution['obs'], solution['act'], pd.DataFrame({'rew': solution['rew']})], axis=1)
    #for fx in ['bp1', 'bp2', 'pump']:
    #  solution[fx] = solution[fx].astype('bool')

    # re-order columns
    solution = solution[state_keys + action_keys + ['rew']]

    # translate element types from integer to string
    for i in range(n_elements):
      k = 'e%i_type'%i
      solution[k] = solution[k].apply(lambda x: element_types[x])

    solution['ei_type'] = solution['ei_type'].apply(lambda x: None if x is None else element_types[x])

    # erase useless designer output after design phase is over
    # solution.loc[env.env.env.env.n_steps_design:, 'ei_type'] = None

    # summarize all the e*_{turbidity,hardness,bacteria,pd,status} columns
    for fx in ['turbidity', 'hardness', 'bacteria', 'pd', 'status']:
      deno = 1 if fx=='status' else 100
      solution['summary_%s'%fx] = np.add.reduce(solution[['e%i_%s'%(i,fx) for i in range(n_elements)]].apply(lambda col: (col//deno).map(str) + ', ', axis=0), axis=1)
      for i in range(n_elements):
        del solution['e%i_%s'%(i,fx)]

    # summarize e*_type
    fx='type'
    solution['summary_%s'%fx] = np.add.reduce(solution[['e%i_%s'%(i,fx) for i in range(n_elements)]].apply(lambda col: col + ', ', axis=0), axis=1)
    for i in range(n_elements):
      del solution['e%i_%s'%(i,fx)]

    return solution


In [0]:

def s_tp1_d2l(s_tp1):
    # convert state_tp1 from dict back to list
    #print("s in:", s_tp1)
    s_tp1 = [s_tp1[k] for k in state_keys]
    return s_tp1


def act_on_wtp(state_t, action_t, debug=False):
  """
  Parameters
  state_t - state values, dict with keys being in `state_keys`
  action_t - action values, dict with keys being in `action_keys`
  
  Returns
  state_tp1 - state at t+1 after action
  reward - reward after action
  water_flowing - true/false if water is flowing
  """
  
  if debug:
    print("-"*20)
    print("act on wtp")
    
  #print("state/act(t)", state_t, action_t)
  
  # convert list to dict
  # state_t = dict(zip(state_keys, state_t))
  
  #print("----------")
  #print(state_t)
  #print("action_t", action_t)

  # initialize
  state_tp1 = state_t.copy()
  reward = 0
  pump_capacity = 10 # pump capacity per time step
  water_flowing = False
  
  # apply cost of installation of element
  if state_t["design_mode"] == 1:
    if element_types[action_t['ei_type']] != 'pipe':
      if debug: print("cost for design change in design mode")
      reward -= 1

  # any action taken
  any_action = any(action_t[x] for x in action_t if x.endswith("_status"))
  if not any_action:
    if debug: print("not doing anything and system is off")
    return state_tp1, reward, water_flowing
  
  # apply energy cost for pump
  if action_t["pump_status"]:
    if debug: print("small punishment for energy consumption: pump")
    reward -= 1
  
  # apply cost reward for energy to take action
  #action_t = dict(zip(action_keys, action_t))
  for i in range(n_elements):
    if action_t["e%i_status"%i] & (element_types[state_t["e%i_type"%i]] != "pipe"):
      if debug: print("small punishment for energy consumption: %s"%element_types[state_t["e%i_type"%i]])
      reward -= 1

  # if pump is off, then nothing is happening
  if not action_t["pump_status"]:
    if debug: print("system is still off")
    return state_tp1, reward, water_flowing
  
  # check blockages
  for i in range(n_elements):
    # if there is a blockage and the element is running (not bypassed)
    if ((state_t["e%i_pd"%i]) == 100):
      if (action_t["e%i_status"%i]):
        if debug: print("the system is stuck at element %i .. halting"%i)
        return state_tp1, reward, water_flowing
      else:
        if debug: print("the system is stuck at element %i .. but it is bypassed"%i)
    
  # from here on, the system is running and not stuck
  if debug: print("pump is on and water is flowing .. proceed")
  water_flowing = True

  # check if water input is enough for pump
  if state_t["wl_in"] >= pump_capacity:
    if debug: print("positive reward for moving water")
    state_tp1["wl_out"] += pump_capacity
    state_tp1["wl_in"] -= pump_capacity
    reward += 7
  else:
    if debug: print("punish since pump will overdraw from raw water tank")
    state_tp1["wl_out"] += state_t["wl_in"]
    state_tp1["wl_in"] = 0
    reward -= 20 # pump burning due to no water
          
  if state_tp1["wl_out"] > 100:
    if debug: print("punish for product tank overflowing")
    state_tp1["wl_out"] = 100
    reward -= 20
    
  # propagate water quality blindly, and later check if improvements were made
  # Notice that e0_hardness will never change
  for i in range(n_elements):
    for el_type_expected in element_uses:
      el_use = element_uses[el_type_expected] # e.g. hardness
      if i < n_elements-1:
        state_tp1["e%i_%s"%(i+1, el_use)] = state_t["e%i_%s"%(i, el_use)]
      else:
        state_tp1["out_%s"%(el_use)] = state_t["e%i_%s"%(i, el_use)]
  
    
  # check water quality improvements
  for i in range(n_elements):
    # if element is "not pipe" and "on"
    el_type = element_types[state_t["e%i_type"%i]]
    if debug: print("Check water quality for '%s' whose status is '%s'"%(el_type, action_t["e%i_status"%i]))
    if (el_type != "pipe") & action_t["e%i_status"%i]:
      el_use = element_uses[el_type] # e.g. hardness
      if debug: print("\tFound non-pipe element %s that is on and that acts on %s ... reward?"%(el_type, el_use))
      # if element's target is in high level
      if state_t["e%i_%s"%(i, el_use)] > 0:
        if debug: print("\treward: element %i is %s lowering %s"%(i, el_type, el_use))
        reward += 5

        # update state space .. next stage will no longer have the high level
        if i < n_elements-1:
          state_tp1["e%i_%s"%(i+1, el_use)] = 0
        else:
          state_tp1["out_%s"%el_use] = 0
          
      else:
        if debug: print("\tpunish: element %i is %s and is not necessary"%(i, el_type))
        reward -= 2
    else:
      if debug: print("\tcase of pipe or bypass")
      for ei in element_uses:
        el_use = element_uses[ei]
        # iterate over all possible targets, and if any of them is high, punish for not doing anything about it
        # if element's target is in high level
        if state_t["e%i_%s"%(i, el_use)] > 0:
          if debug: print("\tpunish: high level of %s but didnt take action"%(el_use))
          reward -= 2
        

  if (state_t['wl_out']<100) & (state_tp1['wl_out']==100):
    if debug: print("bonus points for first attainment of full product tank")
    reward += 10

    # upon getting full, get more bonus points if clean
    all_treated = all(state_tp1[x]==0 for x in state_tp1 if x.startswith("out_"))
    if all_treated:
      if debug: print("bonus points for product tank being totally clean")
      reward += 10

  # done
  return state_tp1, reward, water_flowing



In [8]:
# test case
s_t = {
  'wl_in': 100,
  'e0_turbidity': 100, 'e0_hardness': 0, 'e0_bacteria': 0, 'e0_pd': 0,
  #'e0_type': element_types.index("sand filter"),
  'e0_type': element_types.index("softener"),
    
  'e1_turbidity': 0, 'e1_hardness': 0, 'e1_bacteria': 0, 'e1_pd': 0,
  'e1_type': element_types.index("pipe"),
  'e2_turbidity': 0, 'e2_hardness': 0, 'e2_bacteria': 0, 'e2_pd': 0,
  'e2_type': element_types.index("pipe"),
  'e3_turbidity': 0, 'e3_hardness': 0, 'e3_bacteria': 0, 'e3_pd': 0,
  'e3_type': element_types.index("pipe"),
  'e4_turbidity': 0, 'e4_hardness': 0, 'e4_bacteria': 0, 'e4_pd': 0,
  'e4_type': element_types.index("pipe"),
    
  'e5_turbidity': 0, 'e5_hardness': 0, 'e5_bacteria': 0, 'e5_pd': 0,
  'e5_type': element_types.index("pipe"),
  'e6_turbidity': 0, 'e6_hardness': 0, 'e6_bacteria': 0, 'e6_pd': 0,
  'e6_type': element_types.index("pipe"),
  'e7_turbidity': 0, 'e7_hardness': 0, 'e7_bacteria': 0, 'e7_pd': 0,
  'e7_type': element_types.index("pipe"),
  'e8_turbidity': 0, 'e8_hardness': 0, 'e8_bacteria': 0, 'e8_pd': 0,
  'e8_type': element_types.index("pipe"),
  'e9_turbidity': 0, 'e9_hardness': 0, 'e9_bacteria': 0, 'e9_pd': 0,
  'e9_type': element_types.index("pipe"),
    
  'wl_out': 0,
  'out_turbidity': 0,
  'out_hardness': 0,
  'out_bacteria': 0,
  'design_mode': 1,
}

a_t = {
  'pump_status': 1,
  'ei_type': 0,
  'e0_status': 1, 'e1_status': 0, 'e2_status': 0, 'e3_status': 0, 'e4_status': 0,
  'e5_status': 0, 'e6_status': 0, 'e7_status': 0, 'e8_status': 0, 'e9_status': 0,
}

s_tp1, reward, water_flowing = act_on_wtp(s_t, a_t, debug=True)

mylists = dict(obs=[s_t, s_tp1], act=[a_t, a_t], rew=[reward, None])
df = convert_lists_ObsActRew_df(mylists)

#print(s_tp1, reward)
#df = pd.DataFrame([s_t, s_tp1])
#df['rew'] = None
#df.loc[-1, 'rew'] = reward

#for i in range(n_elements):
#  k = 'e%i_type'%i
#  df[k] = df[k].apply(lambda x: element_types[x])
  
#print(reward)

print('water flowing', water_flowing)

df
#assert reward > 0

--------------------
act on wtp
small punishment for energy consumption: pump
small punishment for energy consumption: softener
pump is on and water is flowing .. proceed
positive reward for moving water
Check water quality for 'softener' whose status is '1'
	Found non-pipe element softener that is on and that acts on hardness ... reward?
	punish: element 0 is softener and is not necessary
Check water quality for 'pipe' whose status is '0'
	case of pipe or bypass
Check water quality for 'pipe' whose status is '0'
	case of pipe or bypass
Check water quality for 'pipe' whose status is '0'
	case of pipe or bypass
Check water quality for 'pipe' whose status is '0'
	case of pipe or bypass
water flowing True


,wl_in,wl_out,out_turbidity,out_hardness,out_bacteria,design_mode,pump_status,ei_type,rew,summary_turbidity,summary_hardness,summary_bacteria,summary_pd,summary_status,summary_type
0,100,0,0,0,0,1,1,pipe,3.0,"1, 0, 0, 0, 0,","0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","1, 0, 0, 0, 0,","softener, pipe, pipe, pipe, pipe,"
1,90,10,0,0,0,1,1,pipe,NaN,"1, 1, 0, 0, 0,","0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","1, 0, 0, 0, 0,","softener, pipe, pipe, pipe, pipe,"


In [9]:
# test case: why pump did not move water
#    wl_in  wl_out  out_turbidity  out_hardness  out_bacteria  pump_status   ei_type   rew summary_turbidity summary_hardness summary_bacteria       summary_pd   summary_status                        summary_type
#      70      30              0             0             0            1      None  -2.0   0, 0, 0, 0, 0,   1, 1, 1, 1, 0,   0, 0, 0, 0, 0,   1, 1, 1, 1, 1,   0, 0, 0, 0, 0,   softener, pipe, pipe, pipe, pipe, 
#      70      30              0             0             0            0      None  -1.0   0, 0, 0, 0, 0,   1, 1, 1, 1, 0,   0, 0, 0, 0, 0,   1, 1, 1, 1, 1,   0, 0, 0, 0, 0,   softener, pipe, pipe, pipe, pipe, 

s_t = {
  'wl_in': 70,
  'wl_out': 30,
  'out_turbidity': 0,
  'out_hardness': 0,
  'out_bacteria': 0,

  #'e0_type': element_types.index("sand filter"),
  'e0_type': element_types.index("softener"),
  'e1_type': element_types.index("pipe"),
  'e2_type': element_types.index("pipe"),
  'e3_type': element_types.index("pipe"),
  'e4_type': element_types.index("pipe"),
    
  'e0_turbidity': 0, 'e0_hardness': 1, 'e0_bacteria': 0, 'e0_pd': 1,
  'e1_turbidity': 0, 'e1_hardness': 1, 'e1_bacteria': 0, 'e1_pd': 1,
  'e2_turbidity': 0, 'e2_hardness': 1, 'e2_bacteria': 0, 'e2_pd': 1,
  'e3_turbidity': 0, 'e3_hardness': 1, 'e3_bacteria': 0, 'e3_pd': 1,
  'e4_turbidity': 0, 'e4_hardness': 0, 'e4_bacteria': 0, 'e4_pd': 1,
    
  'design_mode': 1,
}

a_t = {
  'pump_status': 1,
  'ei_type': 0,
  'e0_status': 0, 'e1_status': 0, 'e2_status': 0, 'e3_status': 0, 'e4_status': 0,
}

s_tp1, reward, water_flowing = act_on_wtp(s_t, a_t, debug=True)

mylists = dict(obs=[s_t, s_tp1], act=[a_t, a_t], rew=[reward, None])
df = convert_lists_ObsActRew_df(mylists)

print('water flowing', water_flowing)

df

--------------------
act on wtp
small punishment for energy consumption: pump
pump is on and water is flowing .. proceed
positive reward for moving water
Check water quality for 'softener' whose status is '0'
	case of pipe or bypass
	punish: high level of hardness but didnt take action
Check water quality for 'pipe' whose status is '0'
	case of pipe or bypass
	punish: high level of hardness but didnt take action
Check water quality for 'pipe' whose status is '0'
	case of pipe or bypass
	punish: high level of hardness but didnt take action
Check water quality for 'pipe' whose status is '0'
	case of pipe or bypass
	punish: high level of hardness but didnt take action
Check water quality for 'pipe' whose status is '0'
	case of pipe or bypass
water flowing True


,wl_in,wl_out,out_turbidity,out_hardness,out_bacteria,design_mode,pump_status,ei_type,rew,summary_turbidity,summary_hardness,summary_bacteria,summary_pd,summary_status,summary_type
0,70,30,0,0,0,1,1,pipe,-2.0,"0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","softener, pipe, pipe, pipe, pipe,"
1,60,40,0,0,0,1,1,pipe,NaN,"0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","0, 0, 0, 0, 0,","softener, pipe, pipe, pipe, pipe,"


# Create gym env

In [0]:
# Create a gym env that simulates the current water treatment plant
# Based on https://github.com/openai/gym/blob/master/gym/envs/toy_text/nchain.py

import gym
from gym import spaces
#from gym.utils import seeding
import numpy as np
import random



In [0]:

# Gym env
class WtpComboEnv_v0(gym.Env):
    """Water Treatment Plant / Combo (operator/design) environment
    
    This is a simulation of operating and designing simultaneously a water treatment plant (WTP).
    
    Observation:
      Parameters in water
      Type: Box(.)
      Water level in .. Range [0, 100]
      Pressure difference on element 1 .. Range [0,10]
      Pressure difference on element 2 .. Range [0,10]
      Water level out .. Range [0, 100]
      
    Actions:
      On/Off commands to pump, by-pass #1, by-pass #2
      Type: Dict of 3x Discrete(3)
      0 Do nothing
      1 flip state from off to on or on to off
      
    Reward: check function "act_on_wtp"
      
    Episode termination:
      Water level out is high at earliest time with minimal by-passing
    """
    def __init__(self):
        # number of steps to complete a WTP design .. same as required number of elements
        # Note that the design phase will spit out 1 element at a time for each position in the WTP
        self.n_steps_design = n_elements

        # number of steps to complete a WTP operation
        self.n_steps_operation = 30 # 10 is the minimum required to achieve full water tank transfer
        
        # number of steps to complete a WTP combo (design + operation)
        self.n_steps = self.n_steps_design + self.n_steps_operation
        
        # number of expected steps to maintain a full product tank without overflowing
        self.n_full_expected = 5
        
        # number of actual steps to maintain a full product tank without overflowing
        self.n_full_actual = 0

        # probability that the differential pressure increases due to blockage
        self.prob_block = 0.5
        
        #  actions: 0 for do nothing, 1 for flip state from off to on or vice versa
        self.action_space = spaces.Tuple([
            # status of pump
            # Note that this action is only active during the 2nd part of the simulation (operation phase)
            spaces.Discrete(2),

            # type of element i
            # Note that this action is only active during the 1st part of the simulation (design phase)
            spaces.Discrete(len(element_types)),

        ] + [
            # status of element i
            # Note that this action is only active during the 2nd part of the simulation (operation phase)
            spaces.Discrete(2),
        ]*n_elements
        )
        
        # observations: water levels, pressure differences, ...
        obs_ranges = [
            [0, 100], # water level in
        ] + [
              # for each element i
              [0, 100], # turbidity
              [0, 100], # hardness
              [0, 100], # bacteria
              [0, 100], # pressure difference
              [0, len(element_types)], # type of element i
          ]*n_elements + [
            [0, 100], # water level out
            [0, 100], # turbidity at output
            [0, 100], # hardness at output
            [0, 100], # bacteria at output
            [0, 1], # design mode
        ]
        
        self.observation_space = spaces.Box(
            low=np.array([x[0] for x in obs_ranges]),
            high=np.array([x[1] for x in obs_ranges]),
            dtype=np.float32
        )
        
        self.reset()
        #self.seed()
        
        # save the initial state for proper reset during simulation
        #  when switching from design to operation
        self.initial_state = None
        
        # debug flag: set to True for verbosity
        self.debug = False
        
    def set_debug(self, debug):
      self.debug = debug

    #def seed(self, seed=None):
    #    self.np_random, seed = seeding.np_random(seed)
    #    return [seed]
    
    def reset(self, s0=None, onlyState=False):
      """
      s0 - desired state
      onlyState - allowing passing this through to continue counting
      """

      if s0 is None:
        # maintain design mode
        designMode = self.state["design_mode"] if onlyState else 1
        
        # sensors and equipment status on/off
        s0 = [100 # water level in
             ] + [# quality of in
                  (100 if (np.random.rand() < 0.25) else 0), # turbidity .. 25% chance of high turbidity
                  (100 if (np.random.rand() < 0.25) else 0), # hardness  .. 25% chance of high level
                  (100 if (np.random.rand() < 0.25) else 0), # bacteria  .. 25% chance of high level
                  0,  # differential pressure
                  0, # type of element .. start with pipe
                 ] + [0,0,0,0,0, # quality at rest of elements + type of element (start with pipe)
                     ]*(n_elements-1
                       ) + [0 # # water level out
                           ] + [0,0,0 # quality at out
                           ] + [designMode # design mode
                               ]
        s0 = dict(zip(state_keys, s0))
        
      # set
      self.state = s0.copy()

      # environment is fully observable, but state space needs to be converted from dict to list
      obs = s_tp1_d2l(self.state)
      
      # early return
      if onlyState: return obs
      
      # other variables
      self.step_i = 0
      self.n_full_actual = 0
      
      return obs
    
    def step(self, act1):
        assert self.action_space.contains(act1), "action not in action space!"
        assert self.step_i < self.n_steps
        
        if self.debug:
          print('-'*20, 'step', self.step_i)

        # first step, save initial state for later
        if self.step_i == 0:
          self.initial_state = self.state.copy()
          
        # if in design mode
        if self.state["design_mode"] == 1:
          # update the element type of the current step, for use in the operation phase
          ei_type = act1[action_keys.index("ei_type")]
          # ei_type = element_types[ei_type]
          self.state["e%i_type"%self.step_i] = ei_type
          self.initial_state["e%i_type"%self.step_i] = ei_type
          
          # force turn on the pump and all elements
          # so that the designer part can find out what's the best design
          # (designer need not know how to operate)
          # Note that in a future iteration, maybe there should be multiple design/operate phases
          # such that maybe the first designer doesn't know how to operate,
          # but later iterations would know how
          act1 = list(act1)
          act1[action_keys.index("pump_status")] = 1
          for i in range(n_elements):
            act1[action_keys.index("e%i_status"%i)] = 1
          act1 = tuple(act1)
        
        # increment number of steps taken
        self.step_i += 1

        # calculate reward of this action
        act2 = dict(zip(action_keys, act1)) # tuple to dict
        self.state, reward, water_flowing = act_on_wtp(self.state, act2, debug=self.debug)
        #print("\t state + element -> state after + reward", wtp_i, self.state, reward_i)
        
        # check if product tank is full
        if self.state['wl_out'] == 100:
          self.n_full_actual += 1
                                      
        # operate the WTP for n_steps, or if the tank is full for x steps
        done = False
        if (self.step_i >= self.n_steps):
          if self.debug: print("simulation done because of max n steps of simulation limit")
          done = True
          
        if (self.n_full_actual >= self.n_full_expected):
          if self.debug: print("simulation done because of max n steps of tank full")
          done = True

        if done:
            # environment is fully observable, but state space needs to be converted from dict to list
            obs_i = s_tp1_d2l(self.state)
            return obs_i, reward, done, {}
          
        # decide on next design mode
        self.state["design_mode"] = 1 if (self.step_i < self.n_steps_design) else 0
          
        # decide on next-state blockage or not
        if self.state["design_mode"] == 1:
          # no blockage during design
          for i in range(n_elements):
            dp_i = "e%i_pd"%i
            self.state[dp_i] = 0
        else:
          for i in range(n_elements):
            blocked_1 = False
            dp_i = "e%i_pd"%i
            if self.state[dp_i] == 0:
              if (element_types[self.state["e%i_type"%i]] != 'pipe') & water_flowing:
                # x% chance of being blocked if not blocked already
                #
                # cannot block a pipe
                # cannot block if water is not flowing
                # Note that if a previous element is blocked, the probability of the current element blocking is halved
                # This avoids all elements blocking at the same time
                blocked_1 = (np.random.rand() < self.prob_block)
            else:
              # already blocked => continue being blocked
              blocked_1 = True

            if blocked_1:
              # 2019-02-14
              ## 50% chance of incrementing by 30 .. note that the blockage is digital at threshold 50
              #dp_increment_1 = (np.random.rand() > 0.5) * 30          
              #self.state[dp_i] += dp_increment_1
              #self.state[dp_i] = min(100, self.state[dp_i]) # cap at 100

              # 2019-02-15 just block straight away
              self.state[dp_i] = 100

        # if switching from design to operation, reset in a special way
        if self.step_i == self.n_steps_design:
          self.reset(self.initial_state, True)

        # environment is fully observable, but state space needs to be converted from dict to list
        obs_i = s_tp1_d2l(self.state)
        return obs_i, reward, done, {}


In [12]:

# iterate
print("smoke test env .. start")
solution = dict(act=[], rew=[], obs=[])

env_test = WtpComboEnv_v0()
env_test.set_debug(True)
obs_t = env_test.reset().copy()

# go through time steps and apply sequence of actions
for t in range(env_test.n_steps + 2):
  obs_t_d = dict(zip(state_keys, obs_t)) # tuple to dict
  solution['obs'].append(obs_t_d) # save trajectory
  
  act_t = env_test.action_space.sample() # random action
  act_t_d = dict(zip(action_keys, act_t)) # tuple to dict
  solution['act'].append(act_t_d) # save action

  obs_t, reward_t, done, _ = env_test.step(act_t)
  solution['rew'].append(reward_t) # save reward

  if done: break
    
  #print("water in", env_test.state, "wtp", [env_test.wtp_elements[x] for x in wtp_i], "water out", water_out, "reward", reward_sum)


print("smoke test env .. end")
len(solution['act'])

smoke test env .. start
-------------------- step 0
--------------------
act on wtp
cost for design change in design mode
small punishment for energy consumption: pump
small punishment for energy consumption: UV
pump is on and water is flowing .. proceed
positive reward for moving water
Check water quality for 'UV' whose status is '1'
	Found non-pipe element UV that is on and that acts on bacteria ... reward?
	punish: element 0 is UV and is not necessary
Check water quality for 'pipe' whose status is '1'
	case of pipe or bypass
Check water quality for 'pipe' whose status is '1'
	case of pipe or bypass
Check water quality for 'pipe' whose status is '1'
	case of pipe or bypass
Check water quality for 'pipe' whose status is '1'
	case of pipe or bypass
-------------------- step 1
--------------------
act on wtp
cost for design change in design mode
small punishment for energy consumption: pump
small punishment for energy consumption: UV
small punishment for energy consumption: sand filter


35

In [13]:

# show result of smoke test
print("*"*30)
print("final observation", dict(zip(state_keys, obs_t)))
print("obs/action sequence chosen")

# mylists = dict(obs=[s_t, s_tp1], act=[a_t, a_t], rew=[reward, None])
print(solution)
df = convert_lists_ObsActRew_df(solution)

"""
df = pd.concat(
  [ 
    pd.DataFrame([dict(zip(state_keys, x)) for x in solution['obs_t']]),
    pd.DataFrame([dict(zip(action_keys, x)) for x in solution['act_t']]),
    pd.DataFrame({"rew": solution['rew']})
  ],
  axis=1
)
df = df[state_keys + action_keys + ['rew']]

# map ei_type fields
k = 'ei_type'
df[k] = df[k].apply(lambda x: element_types[x])
for i in range(n_elements):
  k = 'e%i_type'%i
  df[k] = df[k].apply(lambda x: element_types[x])
"""

with pd.option_context(
    'display.max_colwidth', 20,
    'expand_frame_repr', False,
    'display.max_rows', 250,
    'display.max_columns', 500,
):
  print(df)

******************************
final observation {'wl_in': 60, 'e0_turbidity': 0, 'e0_hardness': 0, 'e0_bacteria': 0, 'e0_pd': 100, 'e0_type': 3, 'e1_turbidity': 0, 'e1_hardness': 0, 'e1_bacteria': 0, 'e1_pd': 100, 'e1_type': 1, 'e2_turbidity': 0, 'e2_hardness': 0, 'e2_bacteria': 0, 'e2_pd': 0, 'e2_type': 0, 'e3_turbidity': 0, 'e3_hardness': 0, 'e3_bacteria': 0, 'e3_pd': 100, 'e3_type': 3, 'e4_turbidity': 0, 'e4_hardness': 0, 'e4_bacteria': 0, 'e4_pd': 0, 'e4_type': 0, 'wl_out': 40, 'out_turbidity': 0, 'out_hardness': 0, 'out_bacteria': 0, 'design_mode': 0}
obs/action sequence chosen
{'act': [{'pump_status': 0, 'ei_type': 3, 'e0_status': 1, 'e1_status': 0, 'e2_status': 1, 'e3_status': 1, 'e4_status': 1}, {'pump_status': 1, 'ei_type': 1, 'e0_status': 1, 'e1_status': 1, 'e2_status': 0, 'e3_status': 0, 'e4_status': 1}, {'pump_status': 0, 'ei_type': 0, 'e0_status': 0, 'e1_status': 0, 'e2_status': 0, 'e3_status': 1, 'e4_status': 0}, {'pump_status': 1, 'ei_type': 3, 'e0_status': 0, 'e1_statu

# Register gym env and train policy

In [0]:
# register the env with gym
# https://github.com/openai/gym/tree/master/gym/envs#how-to-create-new-environments-for-gym
from gym.envs.registration import register

register(
    id='WtpComboEnv-v0',
    #entry_point='gym_foo.envs:FooEnv',
    entry_point=WtpComboEnv_v0,
)

# test registration was successful
env = gym.make("WtpComboEnv-v0")

In [0]:
# The contents of this cell are mostly copied from garage/examples/...
# NEED TO run this twice for the first run in the runtime in colab, 1st for creating the personal config

from garage.baselines import LinearFeatureBaseline # <<<<<< requires restarting the runtime in colab after the 1st dependency installation above
from garage.envs import normalize
#from garage.envs.box2d import CartpoleEnv # no need since will use WtpDesignerEnv_v0 defined above
# from garage.experiment import run_experiment

from garage.tf.algos import TRPO
#from garage.tf.algos import PPO

from garage.tf.envs import TfEnv
#from garage.tf.policies import GaussianMLPPolicy
from garage.tf.policies import CategoricalMLPPolicy

import gym # already imported before

In [16]:
# FOR CPU OR GPU, use this, otherwise for TPU use the below
# ---------------------------------------------------------
# start a tensorflow session so that we can keep it open after training and use the trained network to see it performing
import tensorflow as tf
sess = tf.InteractiveSession()

# no need to initialize
#sess.run(tf.global_variables_initializer())


# check that we're indeed using GPU
# https://stackoverflow.com/a/38019608/4126114
#import tensorflow as tf
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.list_devices()

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 5887640325381631464),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3912983978275719659),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 7061087550218945957),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 11276946637, 12236965970384552454)]

In [0]:
def singlebase_to_multibase(y, dims):
  """
  Illustration
  
  >>> import itertools
  >>> x=np.array(list(itertools.product(range(4), range(3), range(2))))
  >>> y=x[:,0]*2*3 + x[:,1]*2 + x[:,2]
  >>> y.sort()
  >>> y
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23])

  Notice how "y" has no duplicates, and hence is a one-to-one mapping from the original matrix "x"

  Now convert y back to x

  >>> z1  = y//(2*3)
  >>> z1b = y %(2*3)
  >>> z2 = z1b//2
  >>> z2b = z1b %2
  >>> z3 = z2b//1
  >>> z = np.array([np.array(z1), np.array(z2), np.array(z3)]).T

  Notice that z == x
  
  
  Example:
  
  >>> import itertools
  >>> x = np.array(list(itertools.product(range(4), range(3), range(2))))
  >>> y = x[:,0]*2*3 + x[:,1]*2 + x[:,2]
  >>> z = singlebase_to_multibase(y, [4,3,2])
  >>> assert z == x
  """
  # calculate weights
  weights = dims[::-1] # reverse
  weights = np.array(weights).cumprod() # cumulative product
  weights = np.roll(weights, 1) # move last entry to first
  weights[0] = 1 # overwrite
  weights = weights[::-1] # reverse  
  # calcalate output
  z0 = []
  z_b = y
  for w in weights:
    z_  = z_b // w
    z0.append(z_)
    z_b = z_b % w
  # return
  z0 = np.array(z0).T
  return z0

###########
# test
import itertools
x = np.array(list(itertools.product(range(4), range(3), range(2))))
y = x[:,0]*2*3 + x[:,1]*2 + x[:,2]
z = singlebase_to_multibase(y, [4,3,2])
assert (z==x).all()

In [0]:
class UnFlattenActTupleWrapper(gym.ActionWrapper):
    """
    UnFlattens a Discrete action into a Tuple action space
    
    Inherits from ActionWrapper
    https://github.com/openai/gym/blob/6497c9f1c6e43066c8945f02ed3ed4d234f45dc1/gym/core.py
    """
    def __init__(self, env):
        super().__init__(env)

        # save action_space dimensions once
        self.dims = [c.n for c in env.action_space.spaces]
        flat_dim = np.array(self.dims).prod()
        print("flattenting action space to single discrete", flat_dim, self.dims)
        self.action_space = spaces.Discrete(flat_dim)

        
    def action(self, action_in):
        """
        convert a flat action into tuple
        
        based on t0-0e FlattenDictWrapper2
        """
        action_out = singlebase_to_multibase(action_in, self.dims)
        action_out = np.array(action_out).astype('int')
        action_out = tuple(action_out)
        return action_out


In [19]:
import numpy as np
# np.array([2, 4, 2, 4, 2, 4, 2, 4, 2, 4, 2, 4, 2, 4, 2, 4, 2, 4, 2, 4, 2]).prod()
np.array([2, 4, 2, 4, 2, 4, 2, 4, 2, 4, ]).prod()
#np.array([4, 4, 4, 4, 4, ]).prod()

32768

In [20]:
# Train the policy (neural network) on the environment
#----------------------------------
from gym import wrappers

# env = TfEnv(normalize(gym.make("CartPole-v0")))
env = gym.make("WtpComboEnv-v0")
env = UnFlattenActTupleWrapper(env)
env = TfEnv(normalize(env))

# Using larger hidden sizes to learn to use the bypass
hidden_sizes=(32, 32)
#hidden_sizes=(64, 64)

policy = CategoricalMLPPolicy(name="policy", env_spec=env.spec, hidden_sizes=hidden_sizes)

baseline = LinearFeatureBaseline(env_spec=env.spec)


flattenting action space to single discrete 256 [2, 4, 2, 2, 2, 2, 2]
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.random.categorical instead.


In [21]:

algo = TRPO(
#algo = PPO(
    env=env,
    policy=policy,
    baseline=baseline,
    batch_size=4000,
    max_path_length=env.env.env.env.n_steps+2, # add 2 since this is just a safety measure
    #n_itr=5, # smoke test
    n_itr=200,
    discount=0.99,
    max_kl_step=0.01,
    plot=False)


Instructions for updating:
Use tf.cast instead.


In [22]:
env.env.env.env.set_debug(False)
algo.train(sess=sess)

2019-02-16 20:46:39 | itr #0 | Obtaining samples...
2019-02-16 20:46:39 | itr #0 | Obtaining samples for iteration 0...


0% [##############################] 100% | ETA: 00:00:00

2019-02-16 20:46:44 | itr #0 | Processing samples...
2019-02-16 20:46:44 | itr #0 | Logging diagnostics...
2019-02-16 20:46:44 | itr #0 | Optimizing policy...
2019-02-16 20:46:44 | itr #0 | Computing loss before



Total time elapsed: 00:00:05


2019-02-16 20:46:45 | itr #0 | Computing KL before
2019-02-16 20:46:45 | itr #0 | Optimizing
2019-02-16 20:46:45 | itr #0 | Start CG optimization: #parameters: 10528, #inputs: 200, #subsample_inputs: 200
2019-02-16 20:46:45 | itr #0 | computing loss before
2019-02-16 20:46:45 | itr #0 | performing update
2019-02-16 20:46:45 | itr #0 | computing gradient
2019-02-16 20:46:45 | itr #0 | gradient computed
2019-02-16 20:46:45 | itr #0 | computing descent direction
2019-02-16 20:46:45 | itr #0 | descent direction computed
2019-02-16 20:46:45 | itr #0 | backtrack iters: 0
2019-02-16 20:46:45 | itr #0 | computing loss after
2019-02-16 20:46:45 | itr #0 | optimization finished
2019-02-16 20:46:45 | itr #0 | Computing KL after
2019-02-16 20:46:45 | itr #0 | Computing loss after
2019-02-16 20:46:46 | itr #0 | Fitting baseline...
2019-02-16 20:46:46 | itr #0 | Saving snapshot...
2019-02-16 20:46:46 | itr #0 | Saved
2019-02-16 20:46:46 | --------------------------  --------------
2019-02-16 20:46:4

...

0% [##############################] 100% | ETA: 00:00:00

2019-02-16 21:03:25 | itr #199 | Processing samples...
2019-02-16 21:03:25 | itr #199 | Logging diagnostics...
2019-02-16 21:03:25 | itr #199 | Optimizing policy...
2019-02-16 21:03:25 | itr #199 | Computing loss before
2019-02-16 21:03:25 | itr #199 | Computing KL before
2019-02-16 21:03:25 | itr #199 | Optimizing
2019-02-16 21:03:25 | itr #199 | Start CG optimization: #parameters: 10528, #inputs: 141, #subsample_inputs: 141
2019-02-16 21:03:25 | itr #199 | computing loss before
2019-02-16 21:03:25 | itr #199 | performing update
2019-02-16 21:03:25 | itr #199 | computing gradient
2019-02-16 21:03:25 | itr #199 | gradient computed
2019-02-16 21:03:25 | itr #199 | computing descent direction



Total time elapsed: 00:00:04


2019-02-16 21:03:25 | itr #199 | descent direction computed
2019-02-16 21:03:25 | itr #199 | backtrack iters: 1
2019-02-16 21:03:25 | itr #199 | computing loss after
2019-02-16 21:03:25 | itr #199 | optimization finished
2019-02-16 21:03:25 | itr #199 | Computing KL after
2019-02-16 21:03:25 | itr #199 | Computing loss after
2019-02-16 21:03:25 | itr #199 | Fitting baseline...
2019-02-16 21:03:25 | itr #199 | Saving snapshot...
2019-02-16 21:03:25 | itr #199 | Saved
2019-02-16 21:03:25 | --------------------------  -------------
2019-02-16 21:03:25 | AverageDiscountedReturn       64.4125
2019-02-16 21:03:25 | AverageReturn                 69.1064
2019-02-16 21:03:25 | Baseline/ExplainedVariance     0.691368
2019-02-16 21:03:25 | Entropy                        1.85304
2019-02-16 21:03:25 | EnvExecTime                    0.762949
2019-02-16 21:03:25 | Iteration                    199
2019-02-16 21:03:25 | ItrTime                        4.78872
2019-02-16 21:03:25 | MaxReturn             

69.1063829787234

# Results

The below shows a sample water input and the steps in the design/control process of the trained agent

In [23]:
# validate with some experiment
env.env.env.env.set_debug(False)
obs_initial = env.reset().copy()
dict(zip(state_keys, obs_initial))

{'design_mode': 1,
 'e0_bacteria': 0,
 'e0_hardness': 100,
 'e0_pd': 0,
 'e0_turbidity': 0,
 'e0_type': 0,
 'e1_bacteria': 0,
 'e1_hardness': 0,
 'e1_pd': 0,
 'e1_turbidity': 0,
 'e1_type': 0,
 'e2_bacteria': 0,
 'e2_hardness': 0,
 'e2_pd': 0,
 'e2_turbidity': 0,
 'e2_type': 0,
 'e3_bacteria': 0,
 'e3_hardness': 0,
 'e3_pd': 0,
 'e3_turbidity': 0,
 'e3_type': 0,
 'e4_bacteria': 0,
 'e4_hardness': 0,
 'e4_pd': 0,
 'e4_turbidity': 0,
 'e4_type': 0,
 'out_bacteria': 0,
 'out_hardness': 0,
 'out_turbidity': 0,
 'wl_in': 100,
 'wl_out': 0}

In [0]:

# go through time steps and apply sequence of actions
done = False
obs_t_l = obs_initial
solution1 = dict(obs = [], rew = [], act = [])
while not done:
  obs_t_d = dict(zip(state_keys, obs_t_l)) # tuple to dict
  #print("obs", obs_t_d)
  solution1['obs'].append(obs_t_d)
  act_t1, prob_t = policy.get_action(obs_t_l)
  act_t2 = env.env.env.action(act_t1) # convert flat to non-flat
  act_t2 = dict(zip(action_keys, act_t2)) # tuple to dict
  #print("action", act_t1, act_t2)
  solution1['act'].append(act_t2)
  obs_t_l, reward_t, done, _ = env.step(act_t1)
  #print("rew", reward_t)
  solution1['rew'].append(reward_t)
  if done: break

In [25]:

# convert to df
solution2 = convert_lists_ObsActRew_df(solution1)

# display
with pd.option_context(
    'display.max_colwidth', 100,
    'expand_frame_repr', False,
    'display.max_rows', 250,
    'display.max_columns', 500,
):
  print(solution2)

    wl_in  wl_out  out_turbidity  out_hardness  out_bacteria  design_mode  pump_status      ei_type   rew summary_turbidity summary_hardness summary_bacteria       summary_pd   summary_status                                summary_type
0     100       0              0             0             0            1            0     softener   9.0   0, 0, 0, 0, 0,   1, 0, 0, 0, 0,   0, 0, 0, 0, 0,   0, 0, 0, 0, 0,   0, 0, 0, 0, 1,               pipe, pipe, pipe, pipe, pipe, 
1      90      10              0             0             0            1            1         pipe  10.0   0, 0, 0, 0, 0,   1, 0, 0, 0, 0,   0, 0, 0, 0, 0,   0, 0, 0, 0, 0,   0, 1, 1, 0, 0,           softener, pipe, pipe, pipe, pipe, 
2      80      20              0             0             0            1            0         pipe  10.0   0, 0, 0, 0, 0,   1, 0, 0, 0, 0,   0, 0, 0, 0, 0,   0, 0, 0, 0, 0,   0, 0, 0, 0, 0,           softener, pipe, pipe, pipe, pipe, 
3      70      30              0             0          